In [3]:
from langchain_groq import ChatGroq
from api import api_key

In [6]:
llm=ChatGroq(
    temperature=0,
    model_name='llama-3.1-70b-versatile',
    groq_api_key=api_key
)

response=llm.invoke("who was the first person to walk on the moon")
print(response.content)

The first person to walk on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [31]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader('https://jobs.nike.com/job/R-40212?from=job%20search%20funnel')
page_data=loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software EngineerHilversum, North HollandBecome a Part of the NIKE, Inc. TeamNIKE, Inc. d

In [32]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Lead Software Engineer",
  "experience": "Not specified",
  "skills": [
    "Distributed/cloud environments (AWS, GCE, Azure)",
    "Building scalable web applications with microservices architecture",
    "Languages like Java, Javascript, Scala",
    "RESTful services and domain-driven design",
    "IaC tools (Terraform, CloudFormation)",
    "NoSQL databases (Cassandra, DynamoDB)",
    "Relational databases (MySQL, Oracle)",
    "Monitoring tools (Splunk, SignalFx, New Relic)",
    "Automated testing within CI/CD pipelines",
    "Scrum and agile principles"
  ],
  "description": "Lead the development of a fault-tolerant, scalable supply chain platform, driving digital transformation. Design and implement RESTful APIs using domain-driven design, ensuring high performance and reliability. Collaborate with engineers, architects, and product teams to develop agile, scalable technical solutions. Ensure code quality through automated testing, continuous integration, an

In [34]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer',
 'experience': 'Not specified',
 'skills': ['Distributed/cloud environments (AWS, GCE, Azure)',
  'Building scalable web applications with microservices architecture',
  'Languages like Java, Javascript, Scala',
  'RESTful services and domain-driven design',
  'IaC tools (Terraform, CloudFormation)',
  'NoSQL databases (Cassandra, DynamoDB)',
  'Relational databases (MySQL, Oracle)',
  'Monitoring tools (Splunk, SignalFx, New Relic)',
  'Automated testing within CI/CD pipelines',
  'Scrum and agile principles'],
 'description': 'Lead the development of a fault-tolerant, scalable supply chain platform, driving digital transformation. Design and implement RESTful APIs using domain-driven design, ensuring high performance and reliability. Collaborate with engineers, architects, and product teams to develop agile, scalable technical solutions. Ensure code quality through automated testing, continuous integration, and code reviews. Maintain the health of t

In [35]:
type(json_res)

dict

In [20]:
import pandas as pd

df=pd.read_csv('my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [22]:
import chromadb

client=chromadb.PersistentClient('vectorDatabase')
collection=client.create_collection(name='porfolio')


UniqueConstraintError: Collection porfolio already exists

In [25]:
import uuid
if not collection.count():
    for _, row in df.iterrows():  # Add parentheses to call iterrows()
        collection.add(documents=[row['Techstack']],  # Wrap document in a list
                       metadatas={"links": row['Links']},
                       ids=[str(uuid.uuid4())]
                      )


In [37]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links':

In [36]:
job=json_res
job['skills']

['Distributed/cloud environments (AWS, GCE, Azure)',
 'Building scalable web applications with microservices architecture',
 'Languages like Java, Javascript, Scala',
 'RESTful services and domain-driven design',
 'IaC tools (Terraform, CloudFormation)',
 'NoSQL databases (Cassandra, DynamoDB)',
 'Relational databases (MySQL, Oracle)',
 'Monitoring tools (Splunk, SignalFx, New Relic)',
 'Automated testing within CI/CD pipelines',
 'Scrum and agile principles']

In [39]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Khalil, a business development executive at Beeba. Beeba is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Beeba 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Beeba's portfolio: {link_list}
        Remember you are Khalil, BDE at Beeba. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Services for Your Supply Chain Platform

Dear Hiring Manager,

I came across your job posting for a Lead Software Engineer, and I'm excited to introduce Beeba, an AI & Software Consulting company that can help you achieve your goals. Our team of experts has extensive experience in designing and implementing scalable, fault-tolerant software solutions that drive digital transformation.

Our expertise aligns perfectly with your requirements, particularly in:

* Building scalable web applications with microservices architecture
* Designing and implementing RESTful APIs using domain-driven design
* Utilizing IaC tools like Terraform and CloudFormation
* Developing automated testing within CI/CD pipelines
* Implementing agile and Scrum principles

We've successfully delivered numerous projects that showcase our capabilities in software development, DevOps, and cloud computing. Some of our notable projects include:

* [https://example.com/java-portfolio](